In [ ]:
import sklearn.datasets
import sklearn.linear_model
import sklearn.model_selection
import sklearn.ensemble
import polars as pl
import polars.selectors as cs

In [ ]:
df = (
    pl.scan_parquet("credit_card_fraud.parquet")
    .select(cs.starts_with("V"), "Class")
    .collect()
)

features = cs.expand_selector(df, cs.starts_with("V"))

train, test = sklearn.model_selection.train_test_split(df)

model = sklearn.ensemble.RandomForestClassifier(max_depth=4, random_state=208)
model = model.fit(train[features], train["Class"])

pl.DataFrame(
    {"y_true": test["Class"], "y_score": model.predict_proba(test[features])[:, 1]}
).write_parquet("scores.parquet")

In [ ]:
d = sklearn.datasets.load_iris()


features = d.feature_names
df = pl.DataFrame(d.data, schema=features).with_columns(
    pl.Series("target", d.target).eq(1)
)

train, test = sklearn.model_selection.train_test_split(
    df, test_size=0.5, random_state=208
)

model = sklearn.linear_model.LogisticRegression().fit(train[features], train["target"])
preds = model.predict_proba(test[features])[:, 1]

print(len(test))
pl.DataFrame({"y_true": test["target"], "y_score": preds}).write_parquet(
    "iris_scores.parquet"
)